# Parte 1

Implante um servidor web em um cluster KUBERNETES com auto-escalamento horizontal automático.

Pede- se duas implantações:
- Cluster Kubernetes usando minikube
- Cluster AWS EKS

Entregas
- Roteiros de implantação e teste
- Códigos comentados
- Telas
- Artigo formato SBC ou IEEE
- Repositório GITHUB

Apresente um problema e discuta criticamente as implantações. Exemplo de título: técnicas de auto-escalonamento. Procure teses sobre load balancing. Na introdução, diga onde está sua contribuição. Máximo de 12 páginas.

## Horizontal Pod Autoscaler - Minikube



https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/

A HorizontalPodAutoscaler (HPA) automatically updates a workload resource (such as a Deployment or StatefulSet), with the aim of automatically scaling the workload to match demand.

Both **Deployment** and **StatefulSet** are types of workload resources used to manage the lifecycle of applications. Stateless applications do not retain any client data (or "state") between requests. Each request from a client is treated as an independent transaction that is unrelated to any previous request. Stateful applications maintain state across client interactions; this means that they remember previous interactions and use this information to influence future interactions.

**Deployment:** Suitable for stateless applications like web servers, APIs, or other services where any instance of the application can handle incoming requests.
**StatefulSet:** Suitable for databases, distributed systems like Kafka or Cassandra, or any application where each instance needs to maintain its state across restarts.

Horizontal scaling means that the response to increased load is to deploy more Pods. Vertical scaling means assigning more resources (for example: memory or CPU) to the Pods that are already running for the workload.

If the load decreases, and the number of Pods is above the configured minimum, the HorizontalPodAutoscaler instructs the workload resource (the Deployment, StatefulSet, or other similar resource) to scale back down.

Apache HTTP Server, commonly referred to as Apache httpd or simply Apache, is an open-source web server software. Apache is widely used by web hosting providers to serve websites and web applications. Apache listens for incoming HTTP requests on specified ports (usually port 80 for HTTP and port 443 for HTTPS) and processes these requests to serve the requested web pages or resources.

Before you begin, you need to have a Kubernetes cluster, and the kubectl command-line tool must be configured to communicate with your cluster. It is recommended to run this tutorial on a cluster with at least two nodes that are not acting as control plane hosts. Your Kubernetes server must be at or later than version 1.23. You also need to use a cluster that has a Metrics Server deployed and configured. 

Segui o tutorial de instalação do minikube no meu PC local.

```sh
alex@alex-inspiron:~$ minikube status
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured

alex@alex-inspiron:~$ kubectl get nodes
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   19m   v1.30.0

alex@alex-inspiron:~$ minikube stop
✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 node stopped.

```

```sh
minikube start --nodes=2

❗  You cannot change the number of nodes for an existing minikube cluster. Please use 'minikube node add' to add nodes to an existing cluster.
```
Metrics server is enabled (I enabled it right after installing), so I don't need to enable it again.

```sh
alex@alex-inspiron:~$ kubectl get nodes
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   27m   v1.30.0
alex@alex-inspiron:~$ minikube node add
😄  Adding node m02 to cluster minikube as [worker]
❗  Cluster was created without any CNI, adding a node to it might cause broken networking.
👍  Starting "minikube-m02" worker node in "minikube" cluster
🚜  Pulling base image v0.0.44 ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.30.0 on Docker 26.1.1 ...
🔎  Verifying Kubernetes components...
🏄  Successfully added m02 to minikube!
alex@alex-inspiron:~$ kubectl get nodes
NAME           STATUS   ROLES           AGE   VERSION
minikube       Ready    control-plane   28m   v1.30.0
minikube-m02   Ready    <none>          13s   v1.30.0
```

You now have a multi-node setup, with `minikube` as the control-plane node and `minikube-m02` as a worker node. In Kubernetes, when you create a node using Minikube, it may not automatically assign the role as `worker` for nodes added to the cluster. Instead, they appear with the role `<none>`. This is typical behavior in Minikube, as it doesn't automatically label nodes with specific roles like `worker` or `control-plane`.

Adicionei mais um node:

```sh
alex@alex-inspiron:~$ minikube node add
😄  Adding node m03 to cluster minikube as [worker]
👍  Starting "minikube-m03" worker node in "minikube" cluster
🚜  Pulling base image v0.0.44 ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.30.0 on Docker 26.1.1 ...
🔎  Verifying Kubernetes components...
🏄  Successfully added m03 to minikube!
alex@alex-inspiron:~$ kubectl get nodes
NAME           STATUS   ROLES           AGE     VERSION
minikube       Ready    control-plane   34m     v1.30.0
minikube-m02   Ready    <none>          6m25s   v1.30.0
minikube-m03   Ready    <none>          6s      v1.30.0
```




### Run and expose php-apache server

Start a Deployment that runs a container using the `hpa-example` image, and expose it as a Service using the following manifest:

application/php-apache.yaml
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: php-apache
spec:
  selector:
    matchLabels:
      run: php-apache
  template:
    metadata:
      labels:
        run: php-apache
    spec:
      containers:
      - name: php-apache
        image: registry.k8s.io/hpa-example
        ports:
        - containerPort: 80
        resources:
          limits:
            cpu: 500m
          requests:
            cpu: 200m
---
apiVersion: v1
kind: Service
metadata:
  name: php-apache
  labels:
    run: php-apache
spec:
  ports:
  - port: 80
  selector:
    run: php-apache

```

This file describes a Kubernetes Deployment and a Service for running a `php-apache` application. This file contains two main sections: the Deployment and the Service.

A Deployment manages a set of replicated Pods for a given application. In this case, it's used to deploy a PHP application running on an Apache server.

- **apiVersion: apps/v1:** Specifies the API version for the Deployment resource.

- **kind: Deployment:** Indicates that this configuration is for a Deployment.

- **metadata:**
  - **name: php-apache:** The name of the Deployment, which will be used to identify it in the cluster.

- **spec:** Defines the desired state and behavior of the Deployment.
  - **selector:** Specifies how to identify the Pods managed by this Deployment.
    - **matchLabels:** The Deployment will manage Pods with the label `run: php-apache`.
  
  - **template:** Defines the Pod template used by the Deployment to create Pods.
    - **metadata:**
      - **labels:** Adds the label `run: php-apache` to the Pods created by this template.
    
    - **spec:**
      - **containers:** Specifies the container settings for the Pods.
        - **name: php-apache:** The name of the container.
        - **image: registry.k8s.io/hpa-example:** The container image to be used. This example uses an image suitable for demonstrating Horizontal Pod Autoscaling (HPA).
        - **ports:** Defines the network ports exposed by the container.
          - **containerPort: 80:** Exposes port 80, which is typically used for HTTP traffic.
        
        - **resources:** Specifies resource requests and limits for the container.
          - **limits:**
            - **cpu: 500m:** Sets a CPU limit of 500 milliCPU (half a CPU core) for the container.
          - **requests:**
            - **cpu: 200m:** Requests 200 milliCPU, indicating the amount of CPU guaranteed to the container.

A Service in Kubernetes is an abstraction that defines a logical set of Pods and a policy by which to access them, usually by a stable network endpoint.

- **apiVersion: v1:** Specifies the API version for the Service resource.

- **kind: Service:** Indicates that this configuration is for a Service.

- **metadata:**
  - **name: php-apache:** The name of the Service.
  - **labels:** The label `run: php-apache` helps identify this Service.

- **spec:**
  - **ports:**
    - **port: 80:** Exposes the Service on port 80.
  
  - **selector:** Defines which Pods this Service targets by matching the label `run: php-apache`.

The Deployment ensures that the specified number of Pods (replicas) are running. Each Pod runs a container using the specified image (`registry.k8s.io/hpa-example`) with the defined resource constraints. It exposes port 80 for HTTP traffic.

The Service provides a stable endpoint to access the Pods managed by the Deployment. It uses a label selector to route traffic to Pods with the `run: php-apache` label, effectively load balancing the requests among them.

## Scale pod deployments with Horizontal Pod Autoscaler - AWS EKS

https://docs.aws.amazon.com/eks/latest/userguide/horizontal-pod-autoscaler.html